![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_XlmRoBertaForQuestionAnswering.ipynb)

# Import OpenVINO XlmRoBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting XlmRoBertaForQuestionAnswering models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XlmRoBertaForQuestionAnswering from XlmRoBertaForQuestionAnswering   and they have to be in `Question Answering` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [deepset/xlm-roberta-base-squad2](https://huggingface.co/deepset/xlm-roberta-base-squad2) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the OVModelForQuestionAnswering model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [16]:
from optimum.intel import OVModelForQuestionAnswering
from transformers import XLMRobertaTokenizer

MODEL_NAME = "deepset/xlm-roberta-base-squad2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

('ov_models/deepset/xlm-roberta-base-squad2/tokenizer_config.json',
 'ov_models/deepset/xlm-roberta-base-squad2/special_tokens_map.json',
 'ov_models/deepset/xlm-roberta-base-squad2/sentencepiece.bpe.model',
 'ov_models/deepset/xlm-roberta-base-squad2/added_tokens.json')

We need to create a dedicated directory to store asset files and move the `sentencepiece.bpe.model` file into it. This ensures proper compatibility with Spark NLP.

In [17]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [18]:
!ls -l {EXPORT_PATH}/assets

total 4952
-rw-r--r-- 1 root root 5069051 Jul  2 19:58 sentencepiece.bpe.model


## Import and Save XlmRoBertaForQuestionAnswering in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 32.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [19]:
from sparknlp.annotator import XlmRoBertaForQuestionAnswering

spanClassifier = XlmRoBertaForQuestionAnswering.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [20]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your XlmRoBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 1089308
drwxr-xr-x 2 root root       4096 Jul  2 19:55 metadata
-rw-r--r-- 1 root root 1110369972 Jul  2 19:55 xlm_roberta_classification_openvino
-rw-r--r-- 1 root root    5069051 Jul  2 19:55 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [22]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import XlmRoBertaForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = XlmRoBertaForQuestionAnswering.load(f"./{EXPORT_PATH}_spark_nlp_openvino") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

data = spark.createDataFrame([
    ("What is the boiling point of water?", "Water boils at 100 degrees Celsius."),
    ("Who is the president of the USA?", "The president of the USA is Joe Biden."),
    ("What color is the sky?", "The sky is blue on a clear day."),
], ["question", "context"])

model = pipeline.fit(data)
result = model.transform(data)

result.select("question", "answer.result").show(3, truncate=False)

+-----------------------------------+----------------+
|question                           |result          |
+-----------------------------------+----------------+
|What is the boiling point of water?|[at 100 degrees]|
|Who is the president of the USA?   |[is Joe Bi]     |
|What color is the sky?             |[is]            |
+-----------------------------------+----------------+



That's it! You can now go wild and use hundreds of `XlmRoBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
